In [1]:
import sys
import os

from pathlib import Path

data_path = Path("../data")
os.environ["VECTORIAN_DEMO_DATA_PATH"] = str(data_path)

sys.path.append("code")
import nbutils
import gold
import json

from vectorian.importers import TextImporter
from vectorian.corpus import Corpus
from tqdm import tqdm

nlp = nbutils.make_nlp()
gold_data = gold.load_data(data_path / "raw_data" / "gold.json")
contextual_embeddings = dict(
    (k, v) for k, v in nbutils.load_embeddings(data_path / "raw_data" / "embeddings.yml").items()
    if v.is_contextual)

def prepare_docs():
    if contextual_embeddings:
        im = TextImporter(nlp, embeddings=list(contextual_embeddings.values()))
    else:
        im = TextImporter(nlp)

    corpus = Corpus(data_path / "processed_data"/ "corpus")

    for x, d in tqdm(gold_data.in_degree(gold_data.nodes), desc="Importing"):
        if d < 1:
            continue

        node = gold_data.nodes[x]

        doc = im(
            node["context"],
            title=node["source"]["book"],
            author=node["source"]["author"],
            extra_metadata={
                'gold_id': node["id"]
            },
            show_progress=False)

        corpus.add_doc(doc)

prepare_docs()

Importing: 100%|█████████████████████████████████████████████████████████████████████████████████████| 120/120 [04:25<00:00,  2.21s/it]
